<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #7: Transfer Object Detection models to Zenodo</h1>
<h3 align="right"><a href="https://colab.research.google.com/github/ocean-data-factory-sweden/kso/blob/master/tutorials/07_Transfer_ML_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></h3>
<h3 align="right">Written by KSO Team</h3>

# Set up KSO requirements

### Install all the requirements

If you are running this notebook in Google Colab, this cell should take ~2 mins and might restart the session. Please run this cell until you get the "Successful installation!" message.

In [ ]:
import sys
import os

# Check if notebook is running in colab
IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    # Clone kso repo and install requirements
    if not os.path.exists("kso"):
        print("Installing all dependencies...")
        !git clone https://github.com/ocean-data-factory-sweden/kso.git
        !pip install -r /content/kso/requirements_colab.txt

    # Enable external widgets and navigate to the kso tutorial folder
    try:
        from google.colab import output

        output.enable_custom_widget_manager()
        os.chdir("kso/tutorials")
    except ImportError:
        pass

# Prepare the dev settings if needed
try:
    if "kso_utils" not in sys.modules:
        sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))
        import kso_utils

        print("Using development version...")
        # Enables testing changes in utils
        %load_ext autoreload
        %autoreload 2
except ImportError:
    print("Installing latest version from PyPI...")
    %pip install -q kso-utils

# Avoid issues with widgets not displaying properly
!jupyter nbextension enable --user --py widgetsnbextension
!jupyter nbextension enable --user --py jupyter_bbox_widget
!jupyter nbextension enable --user --py ipysheet

# Load the clear output function to keep things clean
from IPython.display import clear_output

clear_output()
print("Successful installation... you're good to go!")

In [ ]:
# Replace nearest neighbours script with custom version (due to relative path issue)
import shutil

shutil.copy(
    "../src/multi_tracker_zoo.py",
    "../yolov5_tracker/trackers/strong_sort/multi_tracker_zoo.py",
)

### Import Python packages

In [ ]:
# Import required modules for tut#7
import kso_utils.widgets as kso_widgets
import kso_utils.project_utils as p_utils
import kso_utils.tutorials_utils as t_utils
import kso_utils.zenodo_utils as zenodo_utils
from kso_utils.project import ProjectProcessor, MLProjectProcessor

print("Packages loaded successfully")

### Choose your project

In [ ]:
project_name = kso_widgets.choose_project()

### Initiate project's database

In [ ]:
# Find project
project = p_utils.find_project(project_name=project_name.value)
# Initialise pp
pp = ProjectProcessor(project)

In [ ]:
# Initialise mlp
mlp = MLProjectProcessor(pp)

# Find model and add to Zenodo

### Choose the model

In [ ]:
model = mlp.choose_model()

### Choose folder to download the model

In [ ]:
download_dir = kso_widgets.choose_folder(".", "downloaded model")

### Download model

In [ ]:
artifact_dir = mlp.get_model(model.value, download_dir.selected)

# ZENODO Upload

### Provide access token to your account

In [ ]:
# It is found in API settings by going to your Zenodo profile and clicking on Applications.
ACCESS_TOKEN = ""

### Find archive for upload to Zenodo

In [ ]:
archive_dir = kso_widgets.choose_folder(".", "archive for upload")

### Add archive to upload

In [ ]:
depo_id = zenodo_utils.upload_archive(ACCESS_TOKEN, artifact_dir=artifact_dir)

### Indicate database title

In [ ]:
upload_title = t_utils.choose_text("title")

### Write database description

In [ ]:
upload_description = t_utils.choose_text("description")

### Indicate database authors together with  their organisations

In [ ]:
authors = t_utils.WidgetMaker()
authors

In [ ]:
authors.author_dict

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Add dataset metadata to upload</font> { vertical-output: true }
zenodo_utils.add_metadata_zenodo_upload(
    ACCESS_TOKEN,
    depo_id,
    upload_title.value,
    upload_description.value,
    authors.author_dict,
)

In [ ]:
# END